In [1]:
import pandas as pd

In [2]:
%config Completer.use_jedi = False

In [34]:
from tqdm._tqdm_notebook import tqdm_notebook

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.


In [3]:
from ast import literal_eval
from nltk.corpus import stopwords

In [4]:
from collections import Counter

In [5]:
from utils.basic_tokenize import tokenize

In [6]:
from gensim.utils import simple_preprocess

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [33]:
party_dict = {28:'conservative',25: 'conservative', 1:'conservative', 46:'quebec', 26: 'conservative',2: 'NDP',5: 'independent',
                4: 'liberal', 9: 'green', 3: 'bloc', 39: 'quebec'}

In [8]:
df = pd.read_csv(r"D:\data\openparliament\df_climate.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
df['text_en']= df.text_en.apply(lambda x: x.replace('Mr. Speaker,',' '))

In [10]:
df['tokens'] = df.tokenized_content.apply(literal_eval)

In [11]:
%%time
df['tokens'] = df.text_en.apply(lambda x: simple_preprocess(x, deacc=True))

Wall time: 5.31 s


In [12]:
stop_words = set(stopwords.words('english'))

In [13]:
df['tokens'] = df.tokens.apply(lambda x: [w for w in x if not w in stop_words])

In [15]:
df.tokens.head()

0    [last, week, six, million, canadians, across, ...
1    [last, week, six, million, canadians, across, ...
2    [last, week, six, million, canadians, across, ...
3    [privilege, able, speak, amendments, bill, pro...
4    [privilege, able, speak, amendments, bill, pro...
Name: tokens, dtype: object

In [16]:
from textblob import TextBlob

In [17]:
blob = TextBlob(df.text_en.iloc[0])

In [18]:
blob.sentiment

Sentiment(polarity=0.3375, subjectivity=0.5791666666666666)

In [19]:
blob.classifier

In [29]:
blob.noun_phrases

WordList(['canadians', "planet 's environment", 'earth', 'bramalea—gore—malton—springdale', 'alternative energies', 'canada', "'s environment", 'federal government', 'sustainable development', 'canada'])

In [27]:
import nltk

In [28]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\alecr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.


True

In [30]:
conservatives = df.politician_id.unique.

In [32]:
df.columns

Index(['index', 'id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'procedural', 'h3_en', 'who_hocid', 'statement_type',
       'written_question', 'source_id', 'who_context_en', 'slug', 'urlcache',
       'wordcount_en', 'tokenized_content', 'slug_length', 'text_en',
       'tokens'],
      dtype='object')

In [36]:
%%time
df['noun_phrases'] = df.text_en.apply(lambda x: TextBlob(x).noun_phrases)

Wall time: 44.5 s


In [38]:
conservative_nps = df[df.party_id==1].noun_phrases

In [39]:
conservative_nps[:10]

3     [bill c-19, environment, environment critic, o...
19    [bush, mid-east oil, north american oil, prime...
25    [president 's, vice-president 's comments, ene...
42    [priorities consist, canadians, canadians, aut...
47    [environmental assessment, constructive sugges...
50    [ndp, canadian jobs, environmental assessment,...
52    [lower churchill, hydro-electric generation pr...
53    [resources, lower churchill, environmental ass...
56               [environmental stewardship, member 's]
57               [environmental stewardship, member 's]
Name: noun_phrases, dtype: object

In [50]:
np_dict = dict()
template_counter = Counter()

In [46]:
parties = df.party_id.unique()

In [48]:
parties

array([ 4,  1, 25, 28,  2,  3, 26,  5, 39,  9, 46], dtype=int64)

In [51]:
def update_np(row):
    if np_dict.get(row.loc['party_id'])==None:
        np_dict[row.loc['party_id']] = template_counter.copy()
        np_dict[row.loc['party_id']].update(row.loc['noun_phrases'])
    else:
        np_dict[row.loc['party_id']].update(row.loc['noun_phrases'])

In [52]:
df.apply(update_np, axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
19655    None
19656    None
19657    None
19658    None
19659    None
Length: 19660, dtype: object

In [59]:
for key,value in np_dict.items():
    print('party: ',party_dict[key])
    print(value.most_common(10))

party:  liberal
[('canada', 3134), ('canadians', 1946), ('prime', 1132), ('kyoto', 910), ('environment', 909), ('conservatives', 779), ('yukon', 555), ('conservative', 520), ('quebec', 481), ('will', 362)]
party:  conservative
[('canada', 4035), ('canadians', 2891), ('carbon tax', 1608), ('greenhouse gas emissions', 1202), ('prime', 1163), ('kyoto', 1017), ('ndp', 931), ('environment', 655), ('greenhouse gases', 611), ('quebec', 579)]
party:  conservative
[('canada', 454), ('kyoto', 346), ('canadians', 304), ('ndp', 153), ('yukon', 116), ('greenhouse gas emissions', 115), ('environment', 110), ('prime', 88), ('carbon tax', 83), ('liberal party', 82)]
party:  conservative
[('canada', 366), ('kyoto', 332), ('canadians', 208), ('ndp', 110), ('greenhouse gas emissions', 109), ('environment', 101), ('liberal party', 75), ('greenhouse gases', 69), ('yukon', 64), ('prime', 61)]
party:  NDP
[('canada', 1822), ('conservatives', 1284), ('canadians', 1161), ('prime', 748), ('environment', 539), (

In [54]:
row = df.iloc[0]

In [55]:
np_dict[row.loc['party_id']].update(row.loc['noun_phrases'])